In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.utils import get_custom_objects
from sklearn.base import BaseEstimator, ClassifierMixin
import joblib

# Register custom metric with Keras
def f1_metric(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1 - y_true) * (1 - y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2 * p * r / (p + r + K.epsilon())
    return K.mean(f1)

get_custom_objects().update({"f1_metric": f1_metric})

# Global Variables
dropout_value = 0.5
epochs_value = 10000
patience_value = 100
random_state_value = 50

# Load the training data
print("Loading training data...")
train_file_path = 'train.csv'  # replace with your actual file path
train_data = pd.read_csv(train_file_path)
print("Training data loaded.")

# Separate features and target variable
X_train = train_data.drop(columns=['Target'])
y_train = train_data['Target']

# Handle missing values
X_train = X_train.fillna(X_train.mean())

# Normalize the features
print("Normalizing features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
print("Features normalized.")

# Build the neural network model
print("Building neural network model...")
def build_nn_model(input_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_value))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_value))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_value))
    model.add(Dense(16, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=[f1_metric])
    return model

# Initialize the neural network model
nn_model = build_nn_model(X_train_scaled.shape[1])

# Train the neural network model
print("Training neural network model...")
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_value, restore_best_weights=True, min_delta=0.001)
model_checkpoint = ModelCheckpoint('best_nn_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, verbose=1)

history = nn_model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=epochs_value, batch_size=32, callbacks=[early_stopping, model_checkpoint, reduce_lr])

# Load the best neural network model
print("Loading best neural network model...")
nn_model.load_weights('best_nn_model.h5')

# Create a wrapper for the neural network model to use it in the StackingClassifier
class NeuralNetworkClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.model = model
        self.classes_ = [0, 1]  # Add the classes_ attribute

    def fit(self, X, y, epochs=100, batch_size=32, verbose=0):
        self.model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=verbose)
        return self

    def predict(self, X):
        return (self.model.predict(X) > 0.5).astype(int).reshape(-1)

    def predict_proba(self, X):
        probas = self.model.predict(X)
        return np.hstack(((1 - probas), probas))

# Initialize the classifiers
print("Initializing classifiers...")
nn_wrapper = NeuralNetworkClassifier(nn_model)

base_classifiers = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=random_state_value)),
    ('knn', KNeighborsClassifier()),
    ('svc', SVC(probability=True, random_state=random_state_value)),
    ('adb', AdaBoostClassifier(random_state=random_state_value)),
    ('gbc', GradientBoostingClassifier(random_state=random_state_value)),
    ('etc', ExtraTreesClassifier(random_state=random_state_value)),
    ('xgb', XGBClassifier(random_state=random_state_value)),
    ('cat', CatBoostClassifier(verbose=0, random_state=random_state_value)),
    ('dt', DecisionTreeClassifier(random_state=random_state_value)),
    ('nn', nn_wrapper)
]

stacking_clf = StackingClassifier(
    estimators=base_classifiers,
    final_estimator=LogisticRegression(),
    cv=5
)

# Train the StackingClassifier on the entire training data
print("Training StackingClassifier on the entire training data...")
stacking_clf.fit(X_train_scaled, y_train)

# Save the StackingClassifier model
print("Saving StackingClassifier model...")
stacking_model_path = "stacking_classifier_model.pkl"
joblib.dump(stacking_clf, stacking_model_path)

# Load the test data
print("Loading test data...")
test_file_path = 'test.csv'  # replace with your actual file path
test_data = pd.read_csv(test_file_path)
print("Test data loaded.")

# Handle missing values in the test data
X_test_final = test_data.fillna(test_data.mean())

# Normalize the test data
print("Normalizing test data...")
X_test_final_scaled = scaler.transform(X_test_final)
print("Test data normalized.")

# Make predictions with the StackingClassifier model
print("Making predictions with the StackingClassifier model...")
y_pred_final = stacking_clf.predict(X_test_final_scaled)

# Save predictions to a CSV file
print("Saving predictions...")
output = pd.DataFrame({'ID': test_data['ID'], 'Target': y_pred_final})
output.to_csv('predictions.csv', index=False)
print("Predictions saved to predictions.csv.")


Loading training data...
Training data loaded.
Normalizing features...
Features normalized.
Building neural network model...
Training neural network model...
Epoch 1/10000
59/67 [=========================>....] - ETA: 0s - loss: 0.9122 - f1_metric: 0.5477 
Epoch 1: val_loss improved from inf to 0.68594, saving model to best_nn_model.h5
67/67 [==============================] - 2s 5ms/step - loss: 0.9001 - f1_metric: 0.5519 - val_loss: 0.6859 - val_f1_metric: 0.6256 - lr: 0.0010
Epoch 2/10000
58/67 [========================>.....] - ETA: 0s - loss: 0.7624 - f1_metric: 0.5922
Epoch 2: val_loss improved from 0.68594 to 0.68243, saving model to best_nn_model.h5


C:\Users\bapti\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


67/67 [==============================] - 0s 3ms/step - loss: 0.7663 - f1_metric: 0.5893 - val_loss: 0.6824 - val_f1_metric: 0.6566 - lr: 0.0010
Epoch 3/10000
61/67 [==========================>...] - ETA: 0s - loss: 0.7548 - f1_metric: 0.6029
Epoch 3: val_loss improved from 0.68243 to 0.68173, saving model to best_nn_model.h5
67/67 [==============================] - 0s 3ms/step - loss: 0.7507 - f1_metric: 0.6086 - val_loss: 0.6817 - val_f1_metric: 0.6631 - lr: 0.0010
Epoch 4/10000
59/67 [=========================>....] - ETA: 0s - loss: 0.7324 - f1_metric: 0.6388
Epoch 4: val_loss improved from 0.68173 to 0.67990, saving model to best_nn_model.h5
67/67 [==============================] - 0s 3ms/step - loss: 0.7285 - f1_metric: 0.6356 - val_loss: 0.6799 - val_f1_metric: 0.6444 - lr: 0.0010
Epoch 5/10000
57/67 [========================>.....] - ETA: 0s - loss: 0.7269 - f1_metric: 0.6262
Epoch 5: val_loss improved from 0.67990 to 0.67903, saving model to best_nn_model.h5
67/67 [============

67/67 [==============================] - 0s 3ms/step - loss: 0.6806 - f1_metric: 0.6822 - val_loss: 0.6796 - val_f1_metric: 0.6832 - lr: 1.0000e-04
Epoch 30/10000
57/67 [========================>.....] - ETA: 0s - loss: 0.6956 - f1_metric: 0.6472
Epoch 30: val_loss did not improve from 0.67784
67/67 [==============================] - 0s 2ms/step - loss: 0.6910 - f1_metric: 0.6590 - val_loss: 0.6790 - val_f1_metric: 0.6892 - lr: 1.0000e-04
Epoch 31/10000
58/67 [========================>.....] - ETA: 0s - loss: 0.6827 - f1_metric: 0.6829
Epoch 31: val_loss did not improve from 0.67784
67/67 [==============================] - 0s 2ms/step - loss: 0.6857 - f1_metric: 0.6828 - val_loss: 0.6786 - val_f1_metric: 0.6928 - lr: 1.0000e-04
Epoch 32/10000
59/67 [=========================>....] - ETA: 0s - loss: 0.6857 - f1_metric: 0.6834
Epoch 32: val_loss did not improve from 0.67784
67/67 [==============================] - 0s 2ms/step - loss: 0.6876 - f1_metric: 0.6790 - val_loss: 0.6790 - val_f1

Epoch 56/10000
59/67 [=========================>....] - ETA: 0s - loss: 0.6754 - f1_metric: 0.6843
Epoch 56: val_loss improved from 0.67456 to 0.67448, saving model to best_nn_model.h5
67/67 [==============================] - 0s 3ms/step - loss: 0.6743 - f1_metric: 0.6878 - val_loss: 0.6745 - val_f1_metric: 0.6909 - lr: 1.0000e-04
Epoch 57/10000
60/67 [=========================>....] - ETA: 0s - loss: 0.6844 - f1_metric: 0.6811
Epoch 57: val_loss did not improve from 0.67448
67/67 [==============================] - 0s 2ms/step - loss: 0.6847 - f1_metric: 0.6834 - val_loss: 0.6746 - val_f1_metric: 0.6909 - lr: 1.0000e-04
Epoch 58/10000
58/67 [========================>.....] - ETA: 0s - loss: 0.6760 - f1_metric: 0.6828
Epoch 58: val_loss improved from 0.67448 to 0.67423, saving model to best_nn_model.h5
67/67 [==============================] - 0s 3ms/step - loss: 0.6777 - f1_metric: 0.6809 - val_loss: 0.6742 - val_f1_metric: 0.6897 - lr: 1.0000e-04
Epoch 59/10000
61/67 [=================

...

Epoch 211/10000
53/67 [======================>.......] - ETA: 0s - loss: 0.6274 - f1_metric: 0.7031
Epoch 211: val_loss did not improve from 0.65163
67/67 [==============================] - 0s 3ms/step - loss: 0.6307 - f1_metric: 0.6989 - val_loss: 0.6522 - val_f1_metric: 0.6943 - lr: 1.0000e-04
Epoch 212/10000
50/67 [=====================>........] - ETA: 0s - loss: 0.6373 - f1_metric: 0.7001
Epoch 212: val_loss did not improve from 0.65163
67/67 [==============================] - 0s 3ms/step - loss: 0.6280 - f1_metric: 0.7154 - val_loss: 0.6523 - val_f1_metric: 0.6935 - lr: 1.0000e-04
Epoch 213/10000
54/67 [=======================>......] - ETA: 0s - loss: 0.6222 - f1_metric: 0.7028
Epoch 213: val_loss did not improve from 0.65163
67/67 [==============================] - 0s 2ms/step - loss: 0.6213 - f1_metric: 0.7104 - val_loss: 0.6525 - val_f1_metric: 0.6914 - lr: 1.0000e-04
Epoch 214/10000
51/67 [=====================>........] - ETA: 0s - loss: 0.6254 - f1_metric: 0.7110
Epoch 214

Epoch 238/10000
60/67 [=========================>....] - ETA: 0s - loss: 0.6187 - f1_metric: 0.7088
Epoch 238: val_loss did not improve from 0.65000
67/67 [==============================] - 0s 2ms/step - loss: 0.6201 - f1_metric: 0.7072 - val_loss: 0.6516 - val_f1_metric: 0.6923 - lr: 1.0000e-04
Epoch 239/10000
59/67 [=========================>....] - ETA: 0s - loss: 0.6095 - f1_metric: 0.7199
Epoch 239: val_loss did not improve from 0.65000
67/67 [==============================] - 0s 2ms/step - loss: 0.6126 - f1_metric: 0.7195 - val_loss: 0.6522 - val_f1_metric: 0.6913 - lr: 1.0000e-04
Epoch 240/10000
58/67 [========================>.....] - ETA: 0s - loss: 0.6063 - f1_metric: 0.7196
Epoch 240: val_loss did not improve from 0.65000
67/67 [==============================] - 0s 2ms/step - loss: 0.6080 - f1_metric: 0.7178 - val_loss: 0.6524 - val_f1_metric: 0.6921 - lr: 1.0000e-04
Epoch 241/10000
60/67 [=========================>....] - ETA: 0s - loss: 0.6166 - f1_metric: 0.7099
Epoch 241

Epoch 266/10000
62/67 [==========================>...] - ETA: 0s - loss: 0.6126 - f1_metric: 0.7167
Epoch 266: val_loss did not improve from 0.65000
67/67 [==============================] - 0s 2ms/step - loss: 0.6094 - f1_metric: 0.7166 - val_loss: 0.6526 - val_f1_metric: 0.6989 - lr: 1.0000e-04
Epoch 267/10000
59/67 [=========================>....] - ETA: 0s - loss: 0.6072 - f1_metric: 0.7249
Epoch 267: val_loss did not improve from 0.65000
67/67 [==============================] - 0s 2ms/step - loss: 0.6068 - f1_metric: 0.7201 - val_loss: 0.6520 - val_f1_metric: 0.6979 - lr: 1.0000e-04
Epoch 268/10000
62/67 [==========================>...] - ETA: 0s - loss: 0.5989 - f1_metric: 0.7289
Epoch 268: val_loss did not improve from 0.65000
67/67 [==============================] - 0s 2ms/step - loss: 0.6025 - f1_metric: 0.7245 - val_loss: 0.6515 - val_f1_metric: 0.6969 - lr: 1.0000e-04
Epoch 269/10000
61/67 [==========================>...] - ETA: 0s - loss: 0.5942 - f1_metric: 0.7299
Epoch 269

Epoch 294/10000
61/67 [==========================>...] - ETA: 0s - loss: 0.5924 - f1_metric: 0.7386
Epoch 294: val_loss did not improve from 0.65000
67/67 [==============================] - 0s 2ms/step - loss: 0.5956 - f1_metric: 0.7364 - val_loss: 0.6525 - val_f1_metric: 0.7048 - lr: 1.0000e-04
Epoch 295/10000
61/67 [==========================>...] - ETA: 0s - loss: 0.5960 - f1_metric: 0.7222
Epoch 295: val_loss did not improve from 0.65000
67/67 [==============================] - 0s 2ms/step - loss: 0.5955 - f1_metric: 0.7216 - val_loss: 0.6526 - val_f1_metric: 0.6993 - lr: 1.0000e-04
Epoch 296/10000
61/67 [==========================>...] - ETA: 0s - loss: 0.5955 - f1_metric: 0.7139
Epoch 296: val_loss did not improve from 0.65000
67/67 [==============================] - 0s 2ms/step - loss: 0.5892 - f1_metric: 0.7210 - val_loss: 0.6526 - val_f1_metric: 0.7038 - lr: 1.0000e-04
Epoch 297/10000
60/67 [=========================>....] - ETA: 0s - loss: 0.5908 - f1_metric: 0.7342
Epoch 297

Epoch 322/10000
56/67 [========================>.....] - ETA: 0s - loss: 0.5850 - f1_metric: 0.7314
Epoch 322: val_loss did not improve from 0.65000
67/67 [==============================] - 0s 2ms/step - loss: 0.5883 - f1_metric: 0.7314 - val_loss: 0.6539 - val_f1_metric: 0.7050 - lr: 1.0000e-04
Epoch 323/10000
55/67 [=======================>......] - ETA: 0s - loss: 0.5747 - f1_metric: 0.7445
Epoch 323: val_loss did not improve from 0.65000
67/67 [==============================] - 0s 2ms/step - loss: 0.5746 - f1_metric: 0.7440 - val_loss: 0.6540 - val_f1_metric: 0.7061 - lr: 1.0000e-04
Epoch 324/10000
57/67 [========================>.....] - ETA: 0s - loss: 0.5767 - f1_metric: 0.7334
Epoch 324: val_loss did not improve from 0.65000
67/67 [==============================] - 0s 2ms/step - loss: 0.5804 - f1_metric: 0.7327 - val_loss: 0.6549 - val_f1_metric: 0.7031 - lr: 1.0000e-04
Epoch 325/10000
60/67 [=========================>....] - ETA: 0s - loss: 0.5798 - f1_metric: 0.7398
Epoch 325